# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Jorge Nozal Martin <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>

 
Problema:
> 1. Sesiones de doblaje <br>

**Descripción del problema:**

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. 

El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:

Número de actores: 10

Número de tomas : 30

Actores/Tomas : https://bit.ly/36D8IuK


(*) La respuesta es obligatoria





                                        

**1.(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>**


In [15]:
!pip install sympy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.3 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.3 MB 1.2 MB/s eta 0:00:05
   ---- ----------------------------------- 0.8/6.3 MB 1.2 MB/s eta 0:00:05
   ------ --------------------------------- 1.0/6.3 MB 1.2 MB/s eta 0:00:05
   -------- ------------------------------- 1.3/6.3 MB 1.2 MB/s eta 0:00:05
   --------- ------------------------------ 1.6/6.3 MB 1.2 MB/s eta 0:00:04
   ----------- ---------------------------- 1.8/6.3 MB 1.2 MB/s eta 0:00:04
   ------------- -------------------------- 2.1/6.3 MB 1.2 MB/s eta 0:00:04
   -------------- ------------------------- 2.4/6.3 MB 1.2 MB/s eta 0:00:04
   ---------------- ----------------------- 2.6/6.3 MB 1.2 MB/s eta 0:00:04
   ------------------ ---------

In [ ]:
#Para calcular el número posible de participaciones sin tener en cuanta resticciones de tomas o actores, podemos usar el número de Bell.
#Es un número que nos da las formas de dividir un cojunto de n elementos en subconjuntos no vacíos (nuestros días de grabación).

from sympy import bell

n = 30
print(f"Número de particiones sin restricciones (número de Bell): B({n}) = {bell(n)}")


Número de particiones sin restricciones (número de Bell): B(30) = 846749014511809332450147


**2.¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.**

In [ ]:
# Como ahora ya tenemos la rstricción de que no se pueden hacer más de 6 tomas al día
# Tenemos que calcular cuántas formas hay de agrupar 30 tomas en grupos de 1 a 6 cada día.

#Usamos una función backtracking

def contar_particiones(n_tomas, max_tomas_por_dia, actual=[]):
    if n_tomas == 0:            #No grabar tomas en un día es posible asi que cuenta 1
        return 1
    
    if n_tomas < 0:             #Si nos pasamos de n_tomas ya no sigue contando participaciones
        return 0

    total = 0                   # Conteo de las particiones válidas

    # Para evitar permutaciones iguales, aseguramos que los bloques estén ordenados de menor a mator 
    start = actual[-1] if actual else 1

    for i in range(start, max_tomas_por_dia + 1):

        total += contar_particiones(n_tomas - i, max_tomas_por_dia, actual + [i])
        
    return total

total_particiones = contar_particiones(30, 6)
print(f"Número de particiones sin repetir orden y con bloques de ≤ 6: {total_particiones}")

#Es posible que la pregunta también se refiera a cuando los actores estan disponibles, pero la restricción en el enunciado es la de 6 tomas por día


Número de particiones sin repetir orden y con bloques de ≤ 6: 1206


Modelo para el espacio de soluciones<br>
**3.(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)**


Respuesta

 Estructura de datos:
 - tomas: dict[int, set[int]] - Se almacenarán actores por toma
    
    Un diccionario donde la clave es el número de la toma (int) y el valor es un conjunto o set de los actores que participan en esa toma (evitado así repetidos).

 - planificación: list[list[int]] - Se almacenarán las tomas por día

    Una lista donde cada elemento es otra lista. Cada sub-lista representa un día de grabación y contiene los números de las tomas (int) que se grabarán ese día

 - asistencia: dict[int, set[int]] - Se almacenarán días por actor

    Un diccionario donde la clave es el identificador del actor (int) y el valor es un conjunto (`set`) de los índices de los días (int) en los que ese actor debe asistir al estudio.

In [2]:
import pandas as pd

# Cargar los datos desde el CSV
ruta = r"./datos_problema_doblaje.csv"
df = pd.read_csv(ruta, header=1) # Usamos header=1 para indicar que la segunda fila es el encabezado, donde aparecen que actores participan

# Eliminar la columna 'Unnamed: 11' y las filas que no son tomas reales
df = df.drop(columns=['Unnamed: 11'], errors='ignore')
df = df[pd.to_numeric(df['Toma'], errors='coerce').notna()]
df = df[df['Toma'] != 'TOTAL']

# Convertir 'Toma' a entero y establecerla como índice
df['Toma'] = df['Toma'].astype(int)
df = df.set_index('Toma')

print(df.head())

        1    2    3    4    5    6    7    8    9   10  Total
Toma                                                         
1     1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0    5.0
2     0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0    3.0
3     0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0    3.0
4     1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0    4.0
5     0.0  1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0    3.0


In [3]:
# Creamos un diccionario para saber qué actores participan en cada toma
tomas = {}

# Solo consideramos columnas que son números
actor_cols = [col for col in df.columns if str(col).isdigit()]
    
for toma_id, row in df.iterrows():
    # Recogemos los id de los actores para la toma
    actores_en_toma = {int(col) for col in actor_cols if row[col] == 1.0}
    tomas[toma_id] = actores_en_toma

# Comprobación de las primeras 5 tomas
for i, (toma_id, actores_set) in enumerate(tomas.items()):
    if i >= 5: 
        break
    print(f"Toma {toma_id}: actores {sorted(list(actores_set))}") 

Toma 1: actores [1, 2, 3, 4, 5]
Toma 2: actores [3, 4, 5]
Toma 3: actores [2, 5, 7]
Toma 4: actores [1, 2, 7, 8]
Toma 5: actores [2, 4, 8]


Según el modelo para el espacio de soluciones<br>

**4.(*)¿Cual es la función objetivo?**

Respuesta

La función objetivo en este problema de optimización es minimizar el gasto total por los servicios de los actores de doblaje.

Dado que los actores cobran la misma cantidad por cada día que deben desplazarse al estudio de grabación (independientemente del número de tomas que graben ese día), el gasto total está directamente relacionado con el número total de días que cada actor asiste al estudio.

def calcular_coste(planificacion, tomas)

El coste total es la suma de la cantidad de días que cada actor debe asistir

In [7]:
from collections import defaultdict

def calcular_coste(planificacion, tomas_data):
    asistencia = defaultdict(set)
    for dia_idx, dia in enumerate(planificacion):
        for toma in dia['tomas']:
            for actor in tomas_data[toma]:
                asistencia[actor].add(dia_idx + 1)  # +1 para días 1-based 
    total_coste = sum(len(dias) for dias in asistencia.values())

    return total_coste

**5.(*)¿Es un problema de maximización o minimización?**

Respuesta

Es un problema de minimización del coste total (días asistidos por actores).

El enunciado del problema indica claramente que el objetivo es "planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible."

Dado que el gasto es el coste y queremos que sea lo más bajo posible, estamos buscando la solución que minimice la función objetivo descripta anteriormente



**6.Diseña un algoritmo para resolver el problema por fuerza bruta**

Respuesta

In [ ]:
from random import sample
from collections import defaultdict

def calcular_coste_fuerza_bruta(planificacion, tomas_data):
    asistencia = defaultdict(set)
    for dia_idx, tomas_dia in enumerate(planificacion, 1):  # Días numerados desde 1
        for toma in tomas_dia:
            for actor in tomas_data[toma]:
                asistencia[actor].add(dia_idx)
    return sum(len(dias) for dias in asistencia.values())

def algoritmo_fuerza_bruta(tomas_data, max_tomas_dia=6, limite=5, iteraciones=100):
    tomas_validas = {k: v for k, v in tomas_data.items() if k in tomas_data}  
    if len(tomas_validas) > limite:
        print(f"Usando solo primeras {limite} tomas de {len(tomas_validas)}")
        tomas_validas = dict(list(tomas_validas.items())[:limite])
    
    mejor_coste = float('inf')
    mejor_plan = None
    tomas_list = list(tomas_validas.keys())
    
    for _ in range(iteraciones):
        perm = sample(tomas_list, len(tomas_list))
        plan = []
        dia_actual = []
        actores_dia = set()
        
        for toma in perm:
            actores_toma = tomas_validas[toma]
            if len(dia_actual) < max_tomas_dia and actores_dia.isdisjoint(actores_toma):
                dia_actual.append(toma)
                actores_dia.update(actores_toma)
            else:
                if dia_actual:
                    plan.append(dia_actual)
                dia_actual = [toma]
                actores_dia = set(actores_toma)
        
        if dia_actual:
            plan.append(dia_actual)
        
        coste = calcular_coste_fuerza_bruta(plan, tomas_validas)
        if coste < mejor_coste:
            mejor_coste = coste
            mejor_plan = plan
    
    return mejor_plan, mejor_coste

# Ejemplo de uso
plan, coste = algoritmo_fuerza_bruta(tomas, max_tomas_dia=6, limite=32, iteraciones=100)
print(f"Días: {len(plan)}, Coste: {coste}")
print("Detalle:")
for idx, dia in enumerate(plan, 1):
    print(f"Día {idx}: Tomas {dia}, Actores {set().union(*[tomas[t] for t in dia])}")

Días: 27, Coste: 94
Detalle:
Día 1: Tomas [23], Actores {1, 3}
Día 2: Tomas [24], Actores {3, 6}
Día 3: Tomas [11], Actores {1, 2, 3, 5, 8}
Día 4: Tomas [17], Actores {1, 3}
Día 5: Tomas [14, 16], Actores {1, 3, 4, 6, 10}
Día 6: Tomas [29], Actores {1, 5, 6}
Día 7: Tomas [3], Actores {2, 5, 7}
Día 8: Tomas [27], Actores {4, 5}
Día 9: Tomas [26, 21], Actores {1, 3, 5, 6, 8, 9}
Día 10: Tomas [15], Actores {1, 2, 7}
Día 11: Tomas [6], Actores {1, 2, 4, 5}
Día 12: Tomas [8], Actores {1, 2, 6}
Día 13: Tomas [1], Actores {1, 2, 3, 4, 5}
Día 14: Tomas [9], Actores {1, 2, 4}
Día 15: Tomas [10], Actores {1, 2, 6, 9}
Día 16: Tomas [18, 25], Actores {1, 2, 3, 4, 6, 10}
Día 17: Tomas [4], Actores {8, 1, 2, 7}
Día 18: Tomas [30], Actores {1, 4}
Día 19: Tomas [19], Actores {1, 3}
Día 20: Tomas [7], Actores {1, 2, 4, 5}
Día 21: Tomas [20], Actores {1, 3, 4, 5}
Día 22: Tomas [5], Actores {8, 2, 4}
Día 23: Tomas [22], Actores {1, 2, 3, 4}
Día 24: Tomas [2], Actores {3, 4, 5}
Día 25: Tomas [13], Actores

**7.Calcula la complejidad del algoritmo por fuerza bruta**

Respuesta

La complejidad del algoritmo por fuerza bruta es muy alta ya que tiene diversos facotres que contribuyen a ella:

- Permutaciones de tomas: Se generan todas las posibles ordenaciones de las tomas (N!).

- Agrupación en días: Para cada permutación, se divide en grupos de hasta 6 tomas.

- Verificación de conflictos: Para cada grupo, se verifica que no haya actores repetidos.

Fórmula de complejidad:

-  O(N!×N×A) = 2.65 x 10e32

Donde:

N = Número de tomas (30)

A = Número máximo de actores por toma (10)

**8.(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta**

Respuesta

In [10]:
def greedy_planificacion(tomas, max_tomas_por_dia=6):
    restantes = set(tomas.keys())
    planificacion = []
    dia_num = 1

    while restantes:
        dia = []
        actores_en_dia = set()
        
        # Ordenar tomas por: 1) actores no usados hoy, 2) tomas con menos actores
        candidatos = sorted(restantes, 
                          key=lambda t: (-len(tomas[t] - actores_en_dia), len(tomas[t])))
        
        for toma in candidatos:
            if len(dia) >= max_tomas_por_dia:
                break
            if actores_en_dia.isdisjoint(tomas[toma]):
                dia.append(toma)
                actores_en_dia.update(tomas[toma])
        
        if not dia:  # Forzar asignación si hay conflictos
            toma = min(restantes, key=lambda t: len(tomas[t] & actores_en_dia))
            dia.append(toma)
            actores_en_dia.update(tomas[toma])
        
        restantes -= set(dia)
        planificacion.append({
            'dia': dia_num,
            'tomas': dia,
            'actores': actores_en_dia,
            'num_tomas': len(dia),
            'num_actores': len(actores_en_dia),
            'tomas_str': ", ".join(map(str, sorted(dia))),
            'actores_str': ", ".join(map(str, sorted(actores_en_dia)))
        })
        dia_num += 1
    
    return planificacion



In [22]:
# Ejecutar el algoritmo
plan = greedy_planificacion(tomas)
coste = calcular_coste(plan, tomas)

print(f"\n{' RESULTADOS ':=^60}")
print(f"Días necesarios: {len(plan)}")
print(f"Coste total (días-actor): {coste}\n")

# Para poder hacer una comprobación en el coste y ver el detalle por día se muestra el "calendario"
print(f"\n{' CALENDARIO ':=^60}")
print("Detalle por día:")
for dia in plan[:5]:
    print(f"\nDía {dia['dia']}:")
    print(f"• Tomas: {dia['tomas_str']}")
    print(f"• Actores: {dia['actores_str']}")
    print(f"• Estadísticas: {dia['num_tomas']} tomas, {dia['num_actores']} actores")


======================== RESULTADOS ========================
Días necesarios: 22
Coste total (días-actor): 94


======================== CALENDARIO ========================
Detalle por día:

Día 1:
• Tomas: 1, 21
• Actores: 1, 2, 3, 4, 5, 6, 8
• Estadísticas: 2 tomas, 7 actores

Día 2:
• Tomas: 11, 16
• Actores: 1, 2, 3, 4, 5, 8, 10
• Estadísticas: 2 tomas, 7 actores

Día 3:
• Tomas: 12
• Actores: 1, 2, 3, 4, 6
• Estadísticas: 1 tomas, 5 actores

Día 4:
• Tomas: 2, 4
• Actores: 1, 2, 3, 4, 5, 7, 8
• Estadísticas: 2 tomas, 7 actores

Día 5:
• Tomas: 6, 18
• Actores: 1, 2, 3, 4, 5, 6
• Estadísticas: 2 tomas, 6 actores


Se trata de una heurística Greedy , por lo que no explora todas las combinacioes posibles sino que toma las mejores decisioes en cada paso, por tanto se reduce masivamente la complejidad.

**9.(*)Calcula la complejidad del algoritmo**

Respuesta

En el peor de los casos este algoritmo deberia tener una complejidad de:

- O(N^2 log N) = 4,410 operaciones aprox

Este será el caso cuando el número total de días sea igual al número total de tomas.En caso de que el número de días sea menor que el de tomas será de:

- O(D X N log N)

Como se puede ver es un orden de magnitud mucho menor que utilizando fuerza bruta

**10.Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios**

Respuesta

In [43]:
import pandas as pd
import random
from collections import defaultdict

def generar_datos_aleatorios(num_tomas=30, num_actores=10, max_actores_por_toma=5,):
    """
    Genera un DataFrame aleatorio con la misma estructura que el problema original.
    
    Args:
        num_tomas (int): Número de tomas a generar (default: 30)
        num_actores (int): Número total de actores (default: 10)
        max_actores_por_toma (int): Máximo de actores por toma (default: 5)
    
    Returns:
        pd.DataFrame: DataFrame con la misma estructura que el CSV original
        dict: Diccionario de tomas (para verificación)
    """
    
    # Generar datos aleatorios
    datos = []
    tomas = defaultdict(set)
    
    for toma_id in range(1, num_tomas + 1):
        # Seleccionar actores aleatorios para esta toma
        num_actores_toma = random.randint(1, max_actores_por_toma)
        actores_toma = random.sample(range(1, num_actores + 1), num_actores_toma)
        
        # Crear fila para el DataFrame
        fila = {'Toma': toma_id}
        for actor in range(1, num_actores + 1):
            fila[str(actor)] = 1.0 if actor in actores_toma else 0.0
        
        datos.append(fila)
        tomas[toma_id] = set(actores_toma)
    
    # Crear DataFrame
    columnas = ['Toma'] + [str(i) for i in range(1, num_actores + 1)] + ['Unnamed: 11']
    df = pd.DataFrame(datos, columns=columnas)
    
    # 3. Añadir fila de TOTALES (opcional)
    total_row = {'Toma': 'TOTAL'}
    for actor in range(1, num_actores + 1):
        total_row[str(actor)] = sum(1 for toma_actors in tomas.values() if actor in toma_actors)
    
    # Convertir total_row a un DataFrame de una sola fila para poder concatenarlo
    total_df = pd.DataFrame([total_row], columns=columnas)
    
    df = pd.concat([df, total_df], ignore_index=True)
    
    return df, tomas

# Ejemplo de uso
datos_aleatorios, tomas_aleatorias = generar_datos_aleatorios(
    num_tomas=30,
    num_actores=10,
    max_actores_por_toma=4,
)

# Mostrar primeras filas
print("Primeras 5 filas del DataFrame generado:")
print(datos_aleatorios.head())

# Verificar diccionario de tomas
print("\nEjemplo de tomas generadas:")
for toma_id in range(1, 6):
    print(f"Toma {toma_id}: Actores {sorted(tomas_aleatorias[toma_id])}")

Primeras 5 filas del DataFrame generado:
  Toma    1    2    3    4    5    6    7    8    9   10  Unnamed: 11
0    1  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0          NaN
1    2  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0          NaN
2    3  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0          NaN
3    4  0.0  0.0  0.0  1.0  0.0  1.0  1.0  1.0  0.0  0.0          NaN
4    5  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0          NaN

Ejemplo de tomas generadas:
Toma 1: Actores [2, 3, 9]
Toma 2: Actores [4, 7]
Toma 3: Actores [2, 4]
Toma 4: Actores [4, 6, 7, 8]
Toma 5: Actores [4]


**11.Aplica el algoritmo al juego de datos generado**

Respuesta

In [41]:
# 1. Primero ejecuta la generación de datos aleatorios (el código que ya tienes)
# [El código de generar_datos_aleatorios que proporcionaste]

# 2. Luego en una nueva celda, aplica el algoritmo greedy:

def greedy_planificacion(tomas, max_tomas_por_dia=6):
    restantes = set(tomas.keys())
    planificacion = []

    while restantes:
        dia = []
        actores_en_dia = set()

        # Ordenar tomas por actores no cubiertos
        candidatos = sorted(restantes, key=lambda t: len(tomas[t] - actores_en_dia), reverse=True)

        for toma in candidatos:
            if len(dia) >= max_tomas_por_dia:
                break
            if actores_en_dia.isdisjoint(tomas[toma]):
                dia.append(toma)
                actores_en_dia.update(tomas[toma])
        
        if not dia:
            toma = restantes.pop()
            dia.append(toma)
        
        restantes -= set(dia)
        planificacion.append(dia)
    
    return planificacion


# Aplicar a los datos generados
plan = greedy_planificacion(tomas_aleatorias)
coste = calcular_coste(plan, tomas_aleatorias)

# Mostrar resultados detallados
print(f"\n{' RESULTADOS ':=^60}")
print(f"Días necesarios: {len(plan)}")
print(f"Coste total (días-actor): {coste}\n")

print("Detalle por día (mostrando primeros 5 días):")
for i, dia in enumerate(plan[:5], 1):
    actores = set().union(*[tomas_aleatorias[t] for t in dia])
    print(f"\nDía {i}:")
    print(f"• Tomas: {dia}")
    print(f"• Actores: {sorted(actores)}")
    print(f"• Estadísticas: {len(dia)} tomas, {len(actores)} actores")

# Verificación
print("\nVerificación:")
print(f"Total tomas asignadas: {sum(len(d) for d in plan)}/{len(tomas_aleatorias)}")
print(f"Actores usados: {len(set().union(*tomas_aleatorias.values()))}")


======================== RESULTADOS ========================
Días necesarios: 11
Coste total (días-actor): 68

Detalle por día (mostrando primeros 5 días):

Día 1:
• Tomas: [9, 6, 22, 23]
• Actores: [1, 2, 3, 4, 5, 6, 7, 8, 9]
• Estadísticas: 4 tomas, 9 actores

Día 2:
• Tomas: [11, 3, 13, 26]
• Actores: [1, 2, 3, 4, 5, 6, 7, 8, 9]
• Estadísticas: 4 tomas, 9 actores

Día 3:
• Tomas: [18, 7, 17]
• Actores: [2, 3, 4, 5, 6, 7, 8, 9, 10]
• Estadísticas: 3 tomas, 9 actores

Día 4:
• Tomas: [27, 2, 24, 12]
• Actores: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
• Estadísticas: 4 tomas, 10 actores

Día 5:
• Tomas: [16, 1, 5, 14, 4]
• Actores: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
• Estadísticas: 5 tomas, 10 actores

Verificación:
Total tomas asignadas: 30/30
Actores usados: 10


**12.Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo**

Respuesta

Apuntes de la asignatura de Algoritmos de Optimización VIU 2025

**13.Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño**

Respuesta

En un estudio real, los actores no ganan todos los mismo así que el primer avance para mí seria tener en cuenta lo que cobra cada actor. No es igual un protagonista que un actor secundario.

Es posible que uno de los actores tenga que sufrir un cambio fisico, por ejemplo el de un náufrago (como Chris Hemsworth en "El corazón del Mar") por lo que muchas de tomas tendran un orden de grabación o dependencia respecto a la fecha. Todas en las que estaba fuerte irán juntas y en las que está muy delgado iran juntas.